# Tools and Tutorials for the analysis of time series data from Kepler/K2 and the TESS missions

Welcome everyone to our TESS webinar!

## Authors

[Rebekah Hounsell](https://heasarc.gsfc.nasa.gov/docs/tess/helpdesk.html) - Support scientist for TESS in the NASA GSFC GI Office. 

## Learning Goals

In this workshop we will teach the user how to access, analyze, and manipulate data from NASA’s exoplanet missions TESS (this can also be applied to Kepler & K2). All tools presented will teach the user how to work with time series data for the purpose of scientific research. 

The workshop assumes a basic knowledge of python and astronomy and will walk the user through several of the concepts outlined below,
- How to obtain TESS data products from the MAST archive
- How to use *LightKurve* to access the various data products and create time series
- How to analyze and assess various data anomalies and how you might visualize them
- How to account for instrumental and noise effects within your data

## Imports

This tutorial requires:
- [**Lightkurve**](https://docs.lightkurve.org) to work with TESS data (v1.11)
- [**Matplotlib**](https://matplotlib.org/) for plotting.
- [**Numpy**](https://numpy.org) for manipulating the data.

In [ ]:
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Introduction: 

The Transiting Exoplanet Survey Satellite (TESS) is a NASA-sponsored Astrophysics Explorer-class mission that is performing a near all-sky survey to search for planets transiting nearby stars. TESS completed its primary mission in July of 2020, and has now entered into its extended mission. The current extended mission will last until September 2022, and will continue to scan the sky for exoplanets and transient events. The TESS mission is now more community focused with larger guest investigator (GI) program.

Over the last three years TESS has observed both the northern and southern hemispheres, with each hemisphere being split into ~13 sectors. The main data products collected by TESS are described below. 

- [Full Frame Images](https://heasarc.gsfc.nasa.gov/docs/tess/data-products.html#full-frame-images): 10 min or 30 min images for each sector. 
- [Target Pixel Files](https://heasarc.gsfc.nasa.gov/docs/tess/data-products.html#target-pixel-files-tpfs): 2 min or 20 second cadenced images of a particular target of interest. 
- [Light Curve Files](https://heasarc.gsfc.nasa.gov/docs/tess/data-products.html#light-curve-files): The flux and time series data produced for each 2 min or 20 second target pixel file object. 

To learn more about the TESS mission and its data products please visit the [TESS GI pages](https://heasarc.gsfc.nasa.gov/docs/tess/).

## 1. How to obtain TESS data products from the MAST archive

You can access the data via several methods 

1. The [ Mikulksi Archive for Space Telescopes (MAST)](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) archive: Here you can enter the name of your object, its TIC number, or an R.A and Dec. 
<video width="900" height="1100" controls src="MAST-recording.mov" />
2. Through the [*Lightkurve*](https://docs.lightkurve.org/tutorials/index.html) package: I will cover this in more detail below.


## 2. How to use *LightKurve* to access the various data products and create a time series

*Lightkurve* offers a user-friendly way to analyze time series data obtained by telescopes, in particular NASA’s Kepler and TESS exoplanet missions. You can search for the various data products for TESS on MAST using the following *Lightkurve* functions.

- To look for your object in a full frame image: [`search_tesscut()`](https://docs.lightkurve.org/api/lightkurve.search.search_tesscut.html)

- To look for target pixel files:  [`search_targetpixelfile()`](https://docs.lightkurve.org/api/lightkurve.search.search_targetpixelfile.html#lightkurve.search.search_targetpixelfile) 

- To obtain light curve files for your object of interest: [`search_lightcurvefile()`](https://docs.lightkurve.org/api/lightkurve.search.search_lightcurvefile.html#lightkurve.search.search_lightcurvefile) 

For the purpose of this tutorial we will be examining [L 98-59](https://arxiv.org/pdf/1903.08017.pdf), a bright M dwarf star at a distance of 10.6 pc. This star is host to three terrestrial-sized planets and is also known in the TESS system as TIC 307210830. 

Lets go through each one of the above functions and see what data is available.

In [ ]:
search_ffi = lk.search_tesscut('TIC 307210830')
search_tpf = lk.search_targetpixelfile('TIC 307210830')
search_lcf = lk.search_lightcurvefile('TIC 307210830')

In [ ]:
search_ffi

The above table indicates that our object was observed in multiple sectors. Note that in the 1st and 2nd Cycle the cadence of the FFI data was 30 min, but in Cycle 3 it is 10 min. Let's see if any other data exists - i.e., was it obseved as a target of interest and so does it have a target pixel file. 

In [ ]:
search_tpf

Great! Our object was observed as a target of interest and has 2 min and 20 ("fast") second cadenced data. This means that there should be light curve files already on the archive. Let's check those out. 

In [ ]:
search_lcf

Wonderful. Light curves for our object of interest have already been created. Let's see what this data looks like by downloading them. 

In [ ]:
lcf = search_lcf.download_all()

In [ ]:
lcf

This has downloaded the light curve for each sector, and stored the data in arrays. You can look at the data for a specific sector by specifying an array number as indicated below. This displays the data for sector 2.

In [ ]:
lcf[0].plot(normalize=True);

Note that there are two different kinds of flux presented in the plot. The **sap_flux** and the **pdcsap_flux**. The definition of each are listed below. 

- **Simple Aperture Photometry (SAP)**:  The SAP light curve is calculated by summing together the brightness of pixels that fall within an aperture set by the *TESS* mission. This is often referred to as the *optimal aperture*, but in spite of its name can sometimes be improved upon! Because the SAP light curve is a sum of the brightness in chosen pixels, it is still subject to systematic artifacts of the mission.

- **Pre-search Data Conditioning SAP flux (PDCSAP) flux**: SAP flux from which long term trends have been removed using so-called Co-trending Basis Vectors (CBVs). PDCSAP flux is usually cleaner data than the SAP flux and will have fewer systematic trends.

You can switch between fluxes using the following commands,

    pdcsap = lcf[0].PDCSAP_FLUX
    
    sapflux = lcf[0].SAP_FLUX

## 3. Understanding data anomalies & noise

Looking at the figure above, you can see that the SAP light curve has a long-term change in brightness that has been removed in the PDCSAP light curve, while keeping the transits at the same depth. For most inspections, a PDCSAP light curve is what you want to use, but when looking at astronomical phenomena that aren't planets (e.g. long-term variability), the SAP flux may be preferred.
    
The primary source of noise removed from the SAP light curve is that of scattered light. Each of TESS's cameras has a lens hood to reduce the scattered light from the Earth and the Moon. Due to TESS's wide field of view and the physical restrictions of the Sun shade the lens hood is not 100% efficient. The effect of the scattered light on the CCD's can be seen in the video below,

<video width="900" height="1100" controls src="ScatteredLight.mov" />

Various method exist to remove this scattered light if a PDCSAP light curve is not readily avalible. We will go into this a bit later, but for now, let's continue to use the PDCSAP flux only. 

In [ ]:
ax = lcf[0].PDCSAP_FLUX.plot() 
ax.set_title("PDCSAP light curve of  L 98-59")

## 4. How to manipulating a light curve
There are a set of useful functions in [`LightCurve`](https://docs.lightkurve.org/api/lightkurve.lightcurve.LightCurve.html#lightkurve.lightcurve.LightCurve) objects which you can use to work with the data. These include:
* [`flatten()`](https://docs.lightkurve.org/api/lightkurve.lightcurve.LightCurve.html#lightkurve.lightcurve.LightCurve.flatten): Remove long term trends using a [Savitzky–Golay filter](https://en.wikipedia.org/wiki/Savitzky%E2%80%93Golay_filter)
* [`remove_outliers()`](https://docs.lightkurve.org/api/lightkurve.lightcurve.LightCurve.html#lightkurve.lightcurve.LightCurve.remove_outliers): Remove outliers using simple sigma clipping
* [`remove_nans()`](https://docs.lightkurve.org/api/lightkurve.lightcurve.LightCurve.html#lightkurve.lightcurve.LightCurve.remove_nans): Remove infinite or NaN values (these can occur during thruster firings)
* [`fold()`](https://docs.lightkurve.org/api/lightkurve.lightcurve.LightCurve.html#lightkurve.lightcurve.LightCurve.fold): Fold the data at a particular period
* [`bin()`](https://docs.lightkurve.org/api/lightkurve.lightcurve.LightCurve.html#lightkurve.lightcurve.LightCurve.bin): Reduce the time resolution of the array, taking the average value in each bin.

We can use these simply on a light curve object

#### Flattening the light curve

In [ ]:
flat_lc = lcf[0].PDCSAP_FLUX.flatten(window_length=401)
flat_lc.plot();

#### Folding the light curve
From the [L 98-59 System](https://arxiv.org/pdf/1903.08017.pdf) paper we know that planet c has a period of 3.690621 days. We can use the `fold()` function to find the transit in our data as shown below.

In [ ]:
folded_lc = flat_lc.fold(period=3.690621)
folded_lc.plot();

### Binning the light curve
Often to see a trend it can be beneficial to bin the data, this can be achieved via the `bin()` function.

In [ ]:
binned_lc = folded_lc.bin(binsize=10)
binned_lc.plot();

Great we can now see our transit very clearly! Note that we can achieve the same plot from our data using one line of code instead of several, see below.

`lcf[0].PDCSAP_FLUX.flatten(window_length=401).fold(period=3.690621).bin(binsize=10).plot();`

## 5. Extracting data from FFI's

What if your object is not a target of interest but simply observed within the full framed images. You can still extract the data and create a 30 min or 10 min cadenced light curve. Lets try to do this with sector 2 data again, note this data is stored in the 2nd array of search_ffi. In this case we also need to specify the size of the region we want to cut out of the FFI to examine.

In [ ]:
ffi_data = search_ffi[1].download(cutout_size=10)

In [ ]:
ffi_data.plot()

The above figure indicates the pixels on the CCD camera, with which L 98-59 was observed. The color indicates the amount of flux in each pixel, in electrons per second. The y-axis shows the pixel row, and the x-axis shows the pixel column. The title tells us the *TESS* Input Catalogue ([TIC](https://tess.mit.edu/science/tess-input-catalogue/)) identification number of the target, and the observing cadence of this image. By default, `plot()` shows the first observation cadence in the Sector.

It looks like our star is isolated, so we can extract a light-curve by simply summing up all the pixel values in each image. To do this we use the [`to_lightcurve`](https://docs.lightkurve.org/api/lightkurve.targetpixelfile.KeplerTargetPixelFile.html#lightkurve.targetpixelfile.KeplerTargetPixelFile.to_lightcurve) function which collects the flux of an object from specified pixels, i.e., an aperture mask. 


To create the SAP lightcurve we have to make our own **aperture mask** for these custom TESS FFI cutouts. Many decisions go into the choice of aperture mask, including the significant blending of the large TESS pixels

In [ ]:
target_mask = ffi_data.create_threshold_mask(threshold=15, reference_pixel='center')
n_target_pixels = target_mask.sum()
n_target_pixels

In [ ]:
ffi_data.plot(aperture_mask=target_mask, mask_color='r');

Nice! We see our target mask centered on the ten brightest pixels in the center of the image. Let’s see what the uncorrected “Simple Aperture Photometry” (SAP) lightcurve looks like. To create our light curve we will pass our **aperture_mask** to the [`to_lightcurve`](https://docs.lightkurve.org/api/lightkurve.targetpixelfile.TessTargetPixelFile.html?highlight=to_lightcurve#lightkurve.targetpixelfile.TessTargetPixelFile.to_lightcurve) function.

In [ ]:
ffi_lc = ffi_data.to_lightcurve(aperture_mask=target_mask)

In [ ]:
ffi_lc.plot(label='Target + background');

Looking at the above lightcurve we see two dominant peaks and can observe that the flux in the aperture is dominated by scattered light. We can tell this because *TESS* orbits Earth twice in each sector, thus patterns which appear twice within a sector are typically related to *TESS’* orbit (such as the scattered light effect).

To remove this light, we are going to detrend the light curve against some vectors which we think are predictive of this systematic noise.

In this case, we can use the pixels outside the aperture as vectors that are highly predictive of the systematic noise, i.e. we will make the assumption that these pixels do not contain any flux from our target.

We can select these pixels very simply by specifying flux outside the aperture using [Python’s bitwise invert operator ~ ](https://wiki.python.org/moin/BitwiseOperators) to take the inverse of the aperture mask, remember that the aperture mask is specified by a boolean array.

In [ ]:
regressors = ffi_data.flux[:, ~target_mask]

In [ ]:
regressors.shape

`regressors` is now an array with shape ntime x npixels outside of the aperture. If we plot the first 10 of these pixels, we can see that they contain mostly scattered light, with some offset terms.

In [ ]:
plt.plot(regressors[:, :10]);

In [linear regression](https://en.wikipedia.org/wiki/Linear_regression) problems, it is common to refer to the matrix of regressors as the design matrix (also known as model matrix or regressor matrix). *Lightkurve* provides a convenient DesignMatrix class which is designed to help you work with detrending vectors.

The DesignMatrix class has several convenience functions, and can be passed into *Lightkurve’s* corrector objects. Please consult the [DesignMatrix page](https://docs.lightkurve.org/api/lightkurve.correctors.DesignMatrix.html) in the API docs for the full details on the methods and features provided by this class.

In [ ]:
dm = lk.DesignMatrix(regressors, name='regressors')
dm

As shown above, dm is now a design matrix with the same shape as the input pixels. Currently, we have 91 pixels that we are using to detrend our light curve against. Rather than using all of the pixels, we can reduce these to their principal components using [Principal Component Analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis). We do this for several reasons:

By reducing to a smaller number of vectors, we can remove some of the stochastic noise in our detrending vectors.

By reducing to the principal components, we can avoid pixels that have intrinsic variability (e.g. from astrophysical long period variables) that can be confused with the true astrophysical signal of our target.
By reducing the number of vectors, our detrending will be faster, (although in this case, this detrending will still take seconds.)

The choice of the number of components is a tricky issue, but in general you should choose a number that is much smaller than the number of vectors.

In [ ]:
dm = dm.pca(5)
dm

Using the pca() method, we have now reduced the number of components in our design matrix to 5. These vectors show a combination of scattered light and other systematic noise, which makes them suited to detrend our input light curve

In [ ]:
plt.plot(ffi_data.time, dm.values + np.arange(5)*0.2, '.')

Note: the DesignMatrix object provides a convenient plot() method to visualize the vectors:

In [ ]:
dm.plot();

We can now de-trend the raw light curve against these vectors. *Lightkurve’s* RegressionCorrector will use linear algebra to find the combination of vectors that makes the input light curve closest to zero. To do this, we need one more component; we need an “offset” term, to be able to fit the mean level of the light curve. We can do this by appending a “constant” to our design matrix.

In [ ]:
dm = dm.append_constant()
dm

Now that we have a design matrix, we only need to pass it into a *Lightkurve*.Corrector. To use our design matrix, we can pass it to the RegressionCorrector, which will de-trend the input light curve against the vectors we’ve built.

Please consult the [RegressionCorrector page](https://docs.lightkurve.org/api/lightkurve.correctors.RegressionCorrector.html?highlight=regressioncorrector#lightkurve.correctors.RegressionCorrector) in the API docs for the full details on the methods and features provided by this class.

In [ ]:
corrector = lk.RegressionCorrector(ffi_lc)
corrector

To correct the light curve, we simply pass in our design matrix.

In [ ]:
corrected_lc = corrector.correct(dm)
ax = ffi_lc.plot(label='Raw light curve')
corrected_lc.plot(ax=ax, label='Corrected light curve');

As shown above, the scattered light from the background has been removed. If we want to see a more in depth look at the correction, we can use the diagnose() method to see what the RegressionCorrector found as the best fitting solution.

In [ ]:
corrector.diagnose();

Regression corrector has clipped out some outliers during the fit of the trend. You can read more about the outlier removal, how to pass a cadence mask, and error propagation in our [docs](https://docs.lightkurve.org/api/lightkurve.correctors.RegressionCorrector.html#lightkurve.correctors.RegressionCorrector).

**Watch Out!**

The RegressionCorrector assumes that you want to remove the trend and set the lightcurve to the mean level of the SAP lightcurve. This isn’t true for *TESS* scattered light. *TESS* FFI lightcurves have additive background, and so we want to reduce the flux to the lowest recorded level, assuming that at that point the contribution from scattered light is approximately zero.

To do this, we will first need to look at the model of the background that RegressionCorrector built. We can access that in the corrector object.

In [ ]:
corrector.model_lc

In [ ]:
model = corrector.model_lc
model.plot();

As you can see above, the model drop below zero flux. This is impossible; the scattered light can’t be removing flux from our target!

To rectify this, we can subtract the model flux value at the 5th percentile.

In [ ]:
model -= np.percentile(model.flux, 5)
model.plot();

This looks better. Now we can simply remove this model from our raw light curve.

In [ ]:
corrected_lc = ffi_lc - model
ax = ffi_lc.plot(label='Raw light curve')
corrected_lc.plot(ax=ax, label='Corrected light curve');

This looks great! Lets summarize this quickly, 

1. Make an aperture mask and a raw light curve

`aper = ffi_data.create_threshold_mask(threshold=15, reference_pixel='center')
ffi_lc = ffi_data.to_lightcurve(aperture_mask=aper)`

2. Make a design matrix and pass it to a linear regression corrector

`dm = lk.DesignMatrix(ffi_data.flux[:, ~aper], name='regressors').pca(5).append_constant()
rc = lk.RegressionCorrector(ffi_lc)
corrected_lc = rc.correct(dm)`

3. Optional: Remove the scattered light, allowing for the large offset from scattered light

`corrected_lc = ffi_lc - rc.model_lc + np.percentile(rc.model_lc.flux, 5)`

Lets now fold and bin the light curve generated here as we did above to return our transit.

In [ ]:
corrected_lc.remove_nans().flatten(window_length=401).fold(period=3.690621).bin(binsize=10).plot();

Excellent we have found our transit again! Remeber the FFI data has a cadence of 30 min.

## Additional Resources 

In this tutorial we have covered the basics of how to obtain, reduce and analize TESS data using *Lightkurve*. We have however only skimmed the surface of what *Lightkurve* can do and how to investigate the data. For more detailed tutorials as well as other useful tools please visit the following pages.

- [*LightKurve Tutorials page*](https://docs.lightkurve.org/tutorials/): A set of 21 tutorilas dealing with Kepler/K2 and TESS data
- [TESS GI data products page](https://heasarc.gsfc.nasa.gov/docs/tess/data-analysis-tools.html): A set of 7 TESS specific tutorials.(Note that these use the devloper version of LightKurve v2.0)
- [STScI Kepler K3 notebooks](https://github.com/spacetelescope/notebooks/tree/master/notebooks/MAST/Kepler): A set of notebooks produced by a collaboration between NumFocus, MAST, *Lightkurve*, and TESS GI office. They make use of python astronomical data packages to demonstrate how to analyze time series data from these NASA missions. New tools are presented here and techniques for the advanced user. (Note that these use the devloper version of LightKurve v2.0)
